In [1]:
import os
import glob
import subprocess
import pandas as pd

In [2]:
DATA_ROOT = "../data/"
SAVE_ROOT = "../processed_data/"

In [3]:
def get_raw_list():
    paths = []
    file_path = SAVE_ROOT + "list_category_8.txt"
    with open(file_path, 'r') as f:
        paths = f.readlines()
    return paths

In [4]:
list_category = get_raw_list()

In [5]:
# filename the input file
def read_file(filename):
    df_raw = pd.read_csv(filename)
    # Select column we want
    df = df_raw.loc[:,['stroke_count','countrycode','drawing_time_total']]
    # Change millisec to sec
    df['drawing_time_total'] = df['drawing_time_total'].div(1000)
    # Change name of column
    df.columns = ['stroke_count','country','drawing_time']
    # Add count per country
    df['count'] = 1
    # Round number
    df = df.groupby('country').agg({'stroke_count': 'mean', 'drawing_time':'mean', 'count':'sum'}).round({'stroke_count':1,'drawing_time':2})
    # Reset index to column for csv file
    df.reset_index(level=0, inplace=True)
    # Count number of occurences by country
    df['word'] = df_raw['word'][0]
    return df

In [6]:
frames = []
for path in list_category:
    raw_id = 'time_per_' + path.split('/')[-1].strip().replace(' ', '_') + '_stats.csv'
    local_path = os.path.join(DATA_ROOT, raw_id)
    df = read_file(local_path)
    frames.append(df)

In [7]:
df_concat = pd.concat(frames, axis=0)

In [8]:
df_concat.to_csv(SAVE_ROOT + 'time_and_stroke.csv')